**Исследование**

# Прогноз

Импорты

In [91]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import  VotingRegressor
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from tqdm.notebook import trange
from tqdm.notebook import tqdm
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
import requests
import warnings


1. Подготовка данных

1. Используйте датасет из Epicurious, подготовленный HugoDarwood
2. Отфильтруйте поля: удалите как можно больше столбцов, которые не относятся к названиями ингридиентов. Вы будете прогнозировать оценку или категорию оценки, используя только ингридиенты и ничего больше. Если модель обучится на других данных, которые не будут передаваться на вход, то точность прогнозов будет заведомо ниже.

In [92]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
df = pd.read_csv('/content/drive/MyDrive/School21/day15/epi_r.csv')

In [94]:
df

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,28.0,2.0,2.0,64.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,671.0,22.0,28.0,583.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,563.0,31.0,38.0,652.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,631.0,45.0,24.0,517.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
# новый список с названием столбцов
new_cols = [col.title() for col in df.columns]
    
# меняем название
df.columns = new_cols
df.columns

Index(['Title', 'Rating', 'Calories', 'Protein', 'Fat', 'Sodium', '#Cakeweek',
       '#Wasteless', '22-Minute Meals', '3-Ingredient Recipes',
       ...
       'Yellow Squash', 'Yogurt', 'Yonkers', 'Yuca', 'Zucchini', 'Cookbooks',
       'Leftovers', 'Snack', 'Snack Week', 'Turkey'],
      dtype='object', length=680)

In [96]:
rec_df = pd.read_json('/content/drive/MyDrive/School21/day15/full_format_recipes.json', orient='columns')
rec_df

,directions,fat,date,categories,calories,desc,protein,rating,title,ingredients,sodium
0,"[1. Place the stock, lentils, celery, carrot, ...",7.0,2006-09-01 04:00:00+00:00,"[Sandwich, Bean, Fruit, Tomato, turkey, Vegeta...",426.0,None,30.0,2.500,"Lentil, Apple, and Turkey Wrap","[4 cups low-sodium vegetable or chicken stock,...",559.0
1,[Combine first 9 ingredients in heavy medium s...,23.0,2004-08-20 04:00:00+00:00,"[Food Processor, Onion, Pork, Bake, Bastille D...",403.0,This uses the same ingredients found in boudin...,18.0,4.375,Boudin Blanc Terrine with Red Onion Confit,"[1 1/2 cups whipping cream, 2 medium onions, c...",1439.0
2,[In a large heavy saucepan cook diced fennel a...,7.0,2004-08-20 04:00:00+00:00,"[Soup/Stew, Dairy, Potato, Vegetable, Fennel, ...",165.0,None,6.0,3.750,Potato and Fennel Soup Hodge,"[1 fennel bulb (sometimes called anise), stalk...",165.0
3,[Heat oil in heavy large skillet over medium-h...,NaN,2009-03-27 04:00:00+00:00,"[Fish, Olive, Tomato, Sauté, Low Fat, Low Cal,...",NaN,The Sicilian-style tomato sauce has tons of Me...,NaN,5.000,Mahi-Mahi in Tomato Olive Sauce,"[2 tablespoons extra-virgin olive oil, 1 cup c...",NaN
4,[Preheat oven to 350°F. Lightly grease 8x8x2-i...,32.0,2004-08-20 04:00:00+00:00,"[Cheese, Dairy, Pasta, Vegetable, Side, Bake, ...",547.0,None,20.0,3.125,Spinach Noodle Casserole,"[1 12-ounce package frozen spinach soufflé, th...",452.0
...,...,...,...,...,...,...,...,...,...,...,...
20125,[Beat whites in a bowl with an electric mixer ...,2.0,2004-08-20 04:00:00+00:00,"[Mixer, Cheese, Egg, Fry, Cocktail Party, Parm...",28.0,None,2.0,3.125,Parmesan Puffs,"[2 large egg whites, 3 oz Parmigiano-Reggiano,...",64.0
20126,[Bring broth to simmer in saucepan.Remove from...,28.0,2008-02-28 22:06:54+00:00,"[Side, Kid-Friendly, High Fiber, Dinner, Parme...",671.0,Cooking the artichokes with the rice infuses t...,22.0,4.375,Artichoke and Parmesan Risotto,"[5 1/2 cups (or more) low-salt chicken broth, ...",583.0
20127,"[Using a sharp knife, cut a shallow X in botto...",38.0,2005-10-21 18:21:20+00:00,"[Onion, Poultry, turkey, Vegetable, Bake, Kid-...",563.0,None,31.0,4.375,Turkey Cream Puff Pie,"[1 small tomato, 1 small onion, finely chopped...",652.0
20128,[Heat 2 tablespoons oil in heavy medium skille...,24.0,2004-08-20 04:00:00+00:00,"[Milk/Cream, Citrus, Dairy, Fish, Garlic, Past...",631.0,"Sharon Hooykaas of Los Alamitos, California, w...",45.0,4.375,Snapper on Angel Hair with Citrus Cream,"[4 tablespoons olive oil, 4 shallots, thinly s...",517.0


In [97]:
# извлечь все категории
categories_initial = []
for l in rec_df.categories:
    if type(l) == list:
        for cat in l:
            if cat not in categories_initial:
                categories_initial.append(cat)
                
# формат
categories_initial = [cat.title() if cat.lower() == cat else cat for cat in categories_initial]                

# сортируем
categories_initial = sorted(categories_initial)
print('First 40 initial categories:', categories_initial[:40])
print()
print('Number of initial categories:', len(categories_initial))

First 40 initial categories: ['#CAKEWEEK', '#WasteLess', '22-Minute Meals', '3-Ingredient Recipes', '30 Days of Groceries', 'Advance Prep Required', 'Alabama', 'Alaska', 'Alcoholic', 'Almond', 'Amaretto', 'Anchovy', 'Anise', 'Anniversary', 'Anthony Bourdain', 'Aperitif', 'Appetizer', 'Apple', 'Apple Juice', 'Apricot', 'Arizona', 'Artichoke', 'Arugula', 'Asian Pear', 'Asparagus', 'Aspen', 'Atlanta', 'Australia', 'Avocado', 'Back to School', 'Backyard BBQ', 'Bacon', 'Bake', 'Banana', 'Barley', 'Basil', 'Bass', 'Bastille Day', 'Bean', 'Beef']

Number of initial categories: 674


In [98]:
categories_initial#все категории

['#CAKEWEEK',
 '#WasteLess',
 '22-Minute Meals',
 '3-Ingredient Recipes',
 '30 Days of Groceries',
 'Advance Prep Required',
 'Alabama',
 'Alaska',
 'Alcoholic',
 'Almond',
 'Amaretto',
 'Anchovy',
 'Anise',
 'Anniversary',
 'Anthony Bourdain',
 'Aperitif',
 'Appetizer',
 'Apple',
 'Apple Juice',
 'Apricot',
 'Arizona',
 'Artichoke',
 'Arugula',
 'Asian Pear',
 'Asparagus',
 'Aspen',
 'Atlanta',
 'Australia',
 'Avocado',
 'Back to School',
 'Backyard BBQ',
 'Bacon',
 'Bake',
 'Banana',
 'Barley',
 'Basil',
 'Bass',
 'Bastille Day',
 'Bean',
 'Beef',
 'Beef Rib',
 'Beef Shank',
 'Beef Tenderloin',
 'Beer',
 'Beet',
 'Bell Pepper',
 'Berry',
 'Beverly Hills',
 'Birthday',
 'Biscuit',
 'Bitters',
 'Blackberry',
 'Blender',
 'Blue Cheese',
 'Blueberry',
 'Boil',
 'Bok Choy',
 'Bon Appétit',
 'Bon App��tit',
 'Boston',
 'Bourbon',
 'Braise',
 'Bran',
 'Brandy',
 'Bread',
 'Breadcrumbs',
 'Breakfast',
 'Brie',
 'Brine',
 'Brisket',
 'Broccoli',
 'Broccoli Rabe',
 'Broil',
 'Brooklyn',
 'Brow

In [99]:
ingredients_initial = []

for i, row in rec_df.iterrows():
    temp_cats = row.categories
    temp_ingr = row.ingredients
    if type(temp_cats) == list and type(temp_ingr) == list:
        for cat in temp_cats:
            cat = cat.lower()
            cat_title = cat.title()
            for ingr in temp_ingr:
                ingr = ingr.lower()
                if ingr.find(cat) != -1 and cat_title not in ingredients_initial:
                    ingredients_initial.append(cat_title)

ingredients_initial = sorted(ingredients_initial)

print('Number of ingredients that are categories:', len(ingredients_initial))

Number of ingredients that are categories: 398


In [100]:
ingredients_initial

['Almond',
 'Amaretto',
 'Anchovy',
 'Anise',
 'Apple',
 'Apple Juice',
 'Apricot',
 'Artichoke',
 'Arugula',
 'Asian Pear',
 'Asparagus',
 'Avocado',
 'Bacon',
 'Bake',
 'Banana',
 'Barley',
 'Basil',
 'Bass',
 'Bean',
 'Beef',
 'Beef Rib',
 'Beef Shank',
 'Beef Tenderloin',
 'Beer',
 'Beet',
 'Bell Pepper',
 'Berry',
 'Bitters',
 'Blackberry',
 'Blender',
 'Blue Cheese',
 'Blueberry',
 'Boil',
 'Bok Choy',
 'Bon Appétit',
 'Bourbon',
 'Braise',
 'Bran',
 'Brandy',
 'Bread',
 'Breadcrumbs',
 'Breakfast',
 'Brie',
 'Brine',
 'Brisket',
 'Broccoli',
 'Broccoli Rabe',
 'Broil',
 'Brown Rice',
 'Brussel Sprout',
 'Buffalo',
 'Bulgur',
 'Butter',
 'Buttermilk',
 'Butternut Squash',
 'Cabbage',
 'Cake',
 'California',
 'Calvados',
 'Campari',
 'Candy',
 'Candy Thermometer',
 'Cantaloupe',
 'Capers',
 'Caraway',
 'Cardamom',
 'Carrot',
 'Cashew',
 'Cauliflower',
 'Celery',
 'Chambord',
 'Champagne',
 'Chard',
 'Chartreuse',
 'Cheddar',
 'Cheese',
 'Cherry',
 'Chestnut',
 'Chicken',
 'Chickpe

In [101]:
not_ingredients = ['Bake', 'Blender', 'Boil', 'Bon Appétit', 'Braise', 'Breakfast', 'Broil', 
                   'California', 'Cocktail', 'Deep-Fry', 'Dessert', 'Dinner', 'Drink', 'Easter', 
                   'Fall', 'Food Processor', 'Fry', 'Fruit', 'Game', 'Gourmet', 'Grill', 'Healthy', 
                   'Ice Cream Machine', 'Juicer', 'Kosher', 'Mandoline', 'Marinate', 'Microwave', 
                   'Mixer', 'Mortar And Pestle', 'New York', 'Passover', 'Pasta Maker', 'Pastry', 
                   'Pie', 'Pizza', 'Poach', 'Pressure Cooker', 'Ramekin', 'Raw', 'Salad', 
                   'Sandwich', 'Sauté', 'Seafood', 'Side', 'Simmer', 'Skewer', 'Slow Cooker', 
                   'Smoker', 'Snack', 'Spring', 'Steam', 'Stew', 'Stir-Fry', 'Summer', 'Tart', 'Turkey',  
                   'Thanksgiving', 'Vegan', 'Vegetarian', 'Winter']

# удаляем не ингредиенты
ingredients = [ingr for ingr in ingredients_initial if ingr not in not_ingredients]

In [102]:
ingredients

['Almond',
 'Amaretto',
 'Anchovy',
 'Anise',
 'Apple',
 'Apple Juice',
 'Apricot',
 'Artichoke',
 'Arugula',
 'Asian Pear',
 'Asparagus',
 'Avocado',
 'Bacon',
 'Banana',
 'Barley',
 'Basil',
 'Bass',
 'Bean',
 'Beef',
 'Beef Rib',
 'Beef Shank',
 'Beef Tenderloin',
 'Beer',
 'Beet',
 'Bell Pepper',
 'Berry',
 'Bitters',
 'Blackberry',
 'Blue Cheese',
 'Blueberry',
 'Bok Choy',
 'Bourbon',
 'Bran',
 'Brandy',
 'Bread',
 'Breadcrumbs',
 'Brie',
 'Brine',
 'Brisket',
 'Broccoli',
 'Broccoli Rabe',
 'Brown Rice',
 'Brussel Sprout',
 'Buffalo',
 'Bulgur',
 'Butter',
 'Buttermilk',
 'Butternut Squash',
 'Cabbage',
 'Cake',
 'Calvados',
 'Campari',
 'Candy',
 'Candy Thermometer',
 'Cantaloupe',
 'Capers',
 'Caraway',
 'Cardamom',
 'Carrot',
 'Cashew',
 'Cauliflower',
 'Celery',
 'Chambord',
 'Champagne',
 'Chard',
 'Chartreuse',
 'Cheddar',
 'Cheese',
 'Cherry',
 'Chestnut',
 'Chicken',
 'Chickpea',
 'Chile',
 'Chile Pepper',
 'Chili',
 'Chill',
 'Chive',
 'Chocolate',
 'Cilantro',
 'Cinnam

In [103]:
#Отфильтруйте поля: удалите как можно больше столбцов, которые не относятся к названиями ингридиентов
cols1 = ['Title', 'Rating']
for cat in ingredients:
    if cat in df.columns:
        cols1.append(cat)

# создаем датафрейм с нужными столбцами
df_narrowed = df[cols1]
df_narrowed

,Title,Rating,Almond,Amaretto,Anchovy,Anise,Apple,Apple Juice,Apricot,Artichoke,...,Whiskey,White Wine,Whole Wheat,Wild Rice,Wine,Wok,Yellow Squash,Yogurt,Yuca,Zucchini
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
df_narrowed.replace([np.inf, -np.inf], np.nan, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [105]:
df_narrowed.fillna(999, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [106]:
df_narrowed.fillna(0, inplace=True)
df_narrowed

,Title,Rating,Almond,Amaretto,Anchovy,Anise,Apple,Apple Juice,Apricot,Artichoke,...,Whiskey,White Wine,Whole Wheat,Wild Rice,Wine,Wok,Yellow Squash,Yogurt,Yuca,Zucchini
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2. Регрессия

1. Для прогнозирования рейтинга опробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе gridsearch и кроссвалидации и оцените RMSE на тестовой подвыборке.
2. Попробуйте различные ансамбли и их гиперпараметры. Выберите лучшее решение на основе gridsearch и кроссвалидации и оцените RMSE на тестовой подвыборке.
3. Посчитайте RMSE для наивного регрессора, в котором вы просто высчитали средний рейтинг.

In [107]:
df_new = df_narrowed.copy()
del df_new['Title']
y = df_new['Rating']
del df_new['Rating']
X = df_new

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state=21, stratify=y)

In [109]:
df_new

,Almond,Amaretto,Anchovy,Anise,Apple,Apple Juice,Apricot,Artichoke,Arugula,Asian Pear,...,Whiskey,White Wine,Whole Wheat,Wild Rice,Wine,Wok,Yellow Squash,Yogurt,Yuca,Zucchini
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
df_new.astype(np.int64)

,Almond,Amaretto,Anchovy,Anise,Apple,Apple Juice,Apricot,Artichoke,Arugula,Asian Pear,...,Whiskey,White Wine,Whole Wheat,Wild Rice,Wine,Wok,Yellow Squash,Yogurt,Yuca,Zucchini
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20048,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
20049,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Линейная регрессия

In [ ]:
params = {'fit_intercept': (True, False)}
lr_grid=GridSearchCV(LinearRegression(), params, n_jobs=-1, scoring = 'neg_mean_squared_error') #поиск лучших параметров
lr_grid.fit(X_train, y_train)
lr_grid.best_params_

{'fit_intercept': True}

In [ ]:
lr_best=LinearRegression(fit_intercept=True)
cross_val_score(lr_best, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-8.515451706517425e+22

Cлучайный лес

In [ ]:
params_rf = {'n_estimators' : [10, 50, 100],
             'max_depth' : [10, 50, 100],
             'max_features' : [10, 50, 100]
             }

In [ ]:
rf_grid=GridSearchCV(RandomForestRegressor(random_state=21), params_rf, cv=10, n_jobs=-1) #поиск лучших параметров
rf_grid.fit(X_train, y_train)
rf_grid.best_params_

{'max_depth': 50, 'max_features': 10, 'n_estimators': 100}

In [ ]:
rf_best=RandomForestRegressor(max_depth=50, max_features=10, n_estimators=100)
cross_val_score(rf_best, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-1.5751260576824322

Дерево решений

In [ ]:
params_tree = {'min_samples_split' : [5, 10, 50],
             'max_depth' : [5, 10, 15],
             'max_features' : [5, 10, 15]
             }

In [ ]:
tree_grid=GridSearchCV(DecisionTreeRegressor(random_state=21), params_tree, cv=10, n_jobs=-1) #поиск лучших параметров
tree_grid.fit(X_train, y_train)
tree_grid.best_params_

{'max_depth': 15, 'max_features': 10, 'min_samples_split': 10}

In [ ]:
tree_best=DecisionTreeRegressor(max_depth=15, max_features=10, min_samples_split=10)
cross_val_score(tree_best, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean()

-1.7552690791560734

Лучшая модель RandomForestRegressor

In [ ]:
rf_best=RandomForestRegressor(max_depth=50, max_features=10, n_estimators=100)
cross_val_score(rf_best, X_test, y_test, cv=10, scoring='neg_mean_squared_error').mean()

-1.6847589959029061

# **Ансамбли**

Voting

In [ ]:
params = {'weights': [(1, 1, 1), (2, 1, 1), (1, 2, 1), (1, 1, 2), (2, 2, 1), (1, 2, 2), (2, 1, 2), (3, 2, 1), (1, 3, 2), (2, 1, 3), (3, 1, 2)],
          'n_jobs': [-1]}

In [ ]:
voting=VotingRegressor(estimators=[('LinearRegression', lr_best),('DecisionTreeRegressor', tree_best),('RandomForestRegressor', rf_best)])
Voti=GridSearchCV(voting,params).fit(X_train, y_train) 


In [ ]:
print ('best parametrs: ',Voti.best_params_)

best parametrs:  {'n_jobs': -1, 'weights': (1, 3, 2)}


In [ ]:
print('RMSE :',cross_val_score(Voti, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

RMSE : -2.3654032518101776e+21


Bagging

In [ ]:
params={
           'n_jobs':[8],
           'n_estimators': (10, 5)
           }

In [ ]:
bc = BaggingRegressor(base_estimator = lr_best)
Baggi = GridSearchCV(bc, params).fit(X_train, y_train) 
print ('best parametrs: ',Baggi.best_params_)
print('RMSE :',cross_val_score(Baggi, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

best parametrs:  {'n_estimators': 5, 'n_jobs': 8}
RMSE : -1.6904316161689611e+22


In [ ]:
bc_tree = BaggingRegressor(base_estimator = tree_best)
Baggi_tree = GridSearchCV(bc_tree, params).fit(X_train, y_train) 
print ('best parametrs: ',Baggi_tree.best_params_)
print('RMSE :',cross_val_score(Baggi_tree, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

best parametrs:  {'n_estimators': 10, 'n_jobs': 8}
RMSE : -1.6867475636543658


In [ ]:
bc_rf = BaggingRegressor(base_estimator = rf_best)
Baggi_rf = GridSearchCV(bc_rf, params).fit(X_train, y_train) 
print ('best parametrs: ',Baggi_rf.best_params_)
print('RMSE :',cross_val_score(Baggi_rf, X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

best parametrs:  {'n_estimators': 5, 'n_jobs': 8}
RMSE : -1.5840406421711086


Лучший ансамбль Bagging на Случайном лесе

In [ ]:
BaggingRegressor_best = BaggingRegressor(base_estimator=rf_best, n_jobs=8,n_estimators=5).fit(X_train, y_train)

In [ ]:

print('RMSE :',cross_val_score(BaggingRegressor_best, X_test, y_test, cv=10, scoring='neg_mean_squared_error').mean())

RMSE : -1.6824251641319716


Наивный регрессор

In [ ]:
import math
from sklearn.metrics import mean_squared_error

In [ ]:
y_mean = y_test.mean()
y_pred = [y_mean for i in range(len(y_test))]
print('RMSE :', math.sqrt(mean_squared_error(y_test, y_pred)))

RMSE : 1.3406047250383304


# 3. Классификация

1. Бинаризируйте значения целевой переменной путем округления рейтингов до ближайшего целого числа. Это и будут ваши классы.
2. Для прогнозирования классов попробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе gridsearch и кроссвалидации и посчитайте accuracy на тестовой подвыборке.
3. Рассчитайте значение accuracy наивного классификатора, в котором вы для всех наблюдений прогнозируете наиболее распространенный класс. Сравните метрики.
4. Снова проведите бинаризацию целевого столбца, преобразовав теперь целые числа в классы bad (0, 1) (невкусное), so-so (2, 3) (нормальное), great (4, 5) (вкусное).
5. Снова для прогнозирования классов попробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение на основе кроссвалидации и посчитайте accuracy на тестовой подвыборке.
6. Снова рассчитайте значение accuracy наивного классификатора, в котором вы для всех наблюдений прогнозируете наиболее распространенный класс. Сравните метрики.
7. Что хуже: спрогнозировать плохой рейтинг, который на самом деле окажется хорошим, или спрогнозировать хороший рейтинг, который на самом деле окажется плохим? Замените метрику accuracy другой соответствующей метрикой.
8. Для прогнозирования классов с новой метрикой опробуйте различные алгоритмы и их гиперпараметры. Выберите лучшее решение и посчитайте метрику на тестовой подвыборке.
9. Попробуйте различные ансамбли и их гиперпараметры. Выберите лучшее решение и посчитайте метрику на тестовой подвыборке.

In [23]:
target = round(y,0).astype(int)

In [24]:
X_t_train, X_t_test, y_t_train, y_t_test = train_test_split(X, target,
                                                    test_size=0.2, random_state=21, stratify=target)

In [25]:
df_new


,Almond,Amaretto,Anchovy,Anise,Apple,Apple Juice,Apricot,Artichoke,Arugula,Asian Pear,...,Whiskey,White Wine,Whole Wheat,Wild Rice,Wine,Wok,Yellow Squash,Yogurt,Yuca,Zucchini
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


LogisticRegression

In [ ]:
lr_params = {'fit_intercept': [True, False],
             'penalty': ['l1', 'l2', 'None'],
             'solver': ['liblinear', 'lbfgs']}

In [ ]:
lr_fit = LogisticRegression(random_state=21)

In [ ]:
lr_grid = GridSearchCV(lr_fit, lr_params, scoring='accuracy', n_jobs=-1)

In [ ]:
lr_grid.fit(X_t_train, y_t_train)
lr_grid.best_params_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: Solver

{'fit_intercept': True, 'penalty': 'l1', 'solver': 'liblinear'}

In [ ]:
lr_best=LogisticRegression(random_state=21, fit_intercept=True, penalty='l1', solver='liblinear')
print('Accuracy :',cross_val_score(lr_best, X_t_train, y_t_train, cv=10, scoring='accuracy').mean())

Accuracy : 0.6672277639235245


SVM

In [ ]:
svm_param = {'kernel': ['linear', 'rbf'],
        'C': [5, 10]}

In [ ]:
svm_fit = SVC(random_state=21, probability=True)

In [ ]:
svm_grid = GridSearchCV(svm_fit, svm_param, scoring='accuracy', n_jobs=-1)

In [ ]:
svm_grid.fit(X_t_train, y_t_train)

GridSearchCV(estimator=SVC(probability=True, random_state=21), n_jobs=-1,
             param_grid={'C': [5, 10], 'kernel': ['linear', 'rbf']},
             scoring='accuracy')

In [ ]:
svm_grid.best_params_

{'C': 5, 'kernel': 'rbf'}

In [ ]:
svm_best=SVC(random_state=21, kernel='rbf', C=5)
print('Accuracy :',cross_val_score(svm_best, X_t_train, y_t_train, cv=10, scoring='accuracy').mean())

Accuracy : 0.6699073966174905


RandomForestClassifier

In [ ]:
rf_params = {'n_estimators': [150, 200, 250],
             'max_depth': [10, 20, 30],
             'criterion': ['gini', 'entropy'],
             'class_weight': ['balanced', None]}

In [ ]:
forest_fit = RandomForestClassifier(random_state=21)

In [ ]:
forest_grid = GridSearchCV(forest_fit, rf_params, scoring='accuracy', n_jobs=-1)

In [ ]:
forest_grid.fit(X_t_train, y_t_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=21), n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20, 30],
                         'n_estimators': [150, 200, 250]},
             scoring='accuracy')

In [ ]:
forest_grid.best_params_

{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 30,
 'n_estimators': 150}

In [ ]:
forest_best=RandomForestClassifier(random_state=21, class_weight=None, criterion='entropy', max_depth=30, n_estimators=150)
print('Accuracy :',cross_val_score(forest_best, X_t_train, y_t_train, cv=10, scoring='accuracy').mean())

Accuracy : 0.6713420887034749


DecisionTree

In [ ]:
dt_param = {'max_depth': range(1, 50),
            'class_weight': ['balanced', None],
            'criterion': ['entropy', 'gini']}

In [ ]:
dt_fit = DecisionTreeClassifier(random_state=21)

In [ ]:
dt_grid = GridSearchCV(dt_fit, dt_param, scoring='accuracy', n_jobs=-1)

In [ ]:
dt_grid.fit(X_t_train, y_t_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=21), n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'criterion': ['entropy', 'gini'],
                         'max_depth': range(1, 50)},
             scoring='accuracy')

In [ ]:
dt_grid.best_params_

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

In [ ]:
dt_best=DecisionTreeClassifier(random_state=21, class_weight=None, criterion='entropy', max_depth=2)
print('Accuracy :',cross_val_score(dt_best, X_t_train, y_t_train, cv=10, scoring='accuracy').mean())

Accuracy : 0.6631755113773199


Лучшая модель RandomForest

In [ ]:
forest_best=RandomForestClassifier(random_state=21, class_weight=None, criterion='entropy', max_depth=30, n_estimators=150)
print('Accuracy :',cross_val_score(forest_best, X_t_test, y_t_test, cv=10, scoring='accuracy').mean())

Accuracy : 0.669407327452513


Наивный классификатор

In [28]:
target.value_counts()

4    13188
5     2719
0     1836
3     1489
2      656
1      164
Name: Rating, dtype: int64

In [29]:
df_naive=pd.DataFrame(y_t_test)

In [ ]:
df_naive['y_pred'] = 4

In [ ]:
print('Accuracy :',accuracy_score(df_naive['Rating'], df_naive['y_pred']))

Accuracy : 0.6576913487908252


Смена таргета

In [26]:
#бинаризацию целевого столбца
cl_ss = target.copy()
cl_ss = cl_ss.replace(0, 'bad')
cl_ss = cl_ss.replace(1, 'bad')
cl_ss = cl_ss.replace(2, 'soso')
cl_ss = cl_ss.replace(3, 'soso')
cl_ss = cl_ss.replace(4, 'great')
cl_ss = cl_ss.replace(5, 'great')
cl_ss

0         soso
1        great
2        great
3        great
4         soso
         ...  
20047     soso
20048    great
20049    great
20050    great
20051    great
Name: Rating, Length: 20052, dtype: object

In [27]:
X_c_train, X_c_test, y_c_train, y_c_test = train_test_split(X, cl_ss,
                                                    test_size=0.2, random_state=21, stratify=cl_ss)

SVM

In [ ]:
svm_c_param = {'kernel': ['linear', 'rbf'],
        'C': [5, 10]}

In [ ]:
svm_c_fit = SVC(random_state=21, probability=True)

In [ ]:
svm_c_grid = GridSearchCV(svm_c_fit, svm_c_param, scoring='accuracy', n_jobs=-1)

In [ ]:
svm_c_grid.fit(X_c_train, y_c_train)

GridSearchCV(estimator=SVC(probability=True, random_state=21), n_jobs=-1,
             param_grid={'C': [5, 10], 'kernel': ['linear', 'rbf']},
             scoring='accuracy')

In [ ]:
svm_c_grid.best_params_

{'C': 5, 'kernel': 'linear'}

In [ ]:
svm_c_best=SVC(random_state=21, kernel='linear', C=5)
print('Accuracy :',cross_val_score(svm_c_best, X_c_train, y_c_train, cv=10, scoring='accuracy').mean())

Accuracy : 0.7984540207114612


DecisionTree

In [ ]:
dt_с_param = {'max_depth': range(1, 50),
            'class_weight': ['balanced', None],
            'criterion': ['entropy', 'gini']}

In [ ]:
dt_с_fit = DecisionTreeClassifier(random_state=21)

In [ ]:
dt_с_grid = GridSearchCV(dt_с_fit, dt_с_param, scoring='accuracy', n_jobs=-1)

In [ ]:
dt_с_grid.fit(X_c_train, y_c_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=21), n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'criterion': ['entropy', 'gini'],
                         'max_depth': range(1, 50)},
             scoring='accuracy')

In [ ]:
dt_с_grid.best_params_

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2}

In [ ]:
dt_с_best=DecisionTreeClassifier(random_state=21, class_weight=None, criterion='gini', max_depth=2)
print('Accuracy :',cross_val_score(dt_с_best, X_c_train, y_c_train, cv=10, scoring='accuracy').mean())

Accuracy : 0.7995761763814763


LogisticRegressor

In [ ]:
lr_c_params = {'fit_intercept': [True, False],
             'penalty': ['l1', 'l2', 'None'],
             'solver': ['liblinear', 'lbfgs']}

In [ ]:
lr_c_fit = LogisticRegression(random_state=21)

In [ ]:
lr_c_grid = GridSearchCV(lr_c_fit, lr_c_params, scoring='accuracy', n_jobs=-1)

In [ ]:
lr_c_grid.fit(X_c_train, y_c_train)
lr_c_grid.best_params_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: Solver

{'fit_intercept': True, 'penalty': 'l2', 'solver': 'liblinear'}

In [ ]:
lr_c_best=LogisticRegression(random_state=21, fit_intercept=True, penalty='l2', solver='liblinear')
print('Accuracy :',cross_val_score(lr_c_best, X_c_train, y_c_train, cv=10, scoring='accuracy').mean())

Accuracy : 0.7998879359234313


RandomForest

In [ ]:
rf_c_params = {'n_estimators': [150, 200, 250],
             'max_depth': [10, 20, 30],
             'criterion': ['gini', 'entropy'],
             'class_weight': ['balanced', None]}

In [ ]:
forest_c_fit = RandomForestClassifier(random_state=21)

In [ ]:
forest_c_grid = GridSearchCV(forest_c_fit, rf_c_params, scoring='accuracy', n_jobs=-1)

In [ ]:
forest_c_grid.fit(X_c_train, y_c_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=21), n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20, 30],
                         'n_estimators': [150, 200, 250]},
             scoring='accuracy')

In [ ]:
forest_c_grid.best_params_

{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 30,
 'n_estimators': 150}

In [ ]:
forest_c_best=RandomForestClassifier(random_state=21, class_weight=None, criterion='entropy', max_depth=30, n_estimators=150)
print('Accuracy :',cross_val_score(forest_c_best, X_c_train, y_c_train, cv=10, scoring='accuracy').mean())

Accuracy : 0.8007605596600399


Лучшая модель RandomForest

In [ ]:
forest_c_best=RandomForestClassifier(random_state=21, class_weight=None, criterion='entropy', max_depth=30, n_estimators=150)
print('Accuracy :',cross_val_score(forest_c_best, X_c_test, y_c_test, cv=10, scoring='accuracy').mean())

Accuracy : 0.7965589756950907


Наивный классификатор

In [ ]:
cl_ss.value_counts()

great    15907
soso      2145
bad       2000
Name: Rating, dtype: int64

In [ ]:
df_c_naive=pd.DataFrame(y_c_test)

In [ ]:
df_c_naive['y_pred'] = 'great'

In [ ]:
print('Accuracy :',accuracy_score(df_c_naive['Rating'], df_c_naive['y_pred']))

Accuracy : 0.7933183744702069


Смена метрики

In [1]:
import warnings
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score

In [ ]:
warnings.filterwarnings('ignore')
forest_c_best=RandomForestClassifier(random_state=21, class_weight=None, criterion='entropy', max_depth=30, n_estimators=150).fit(X_c_train, y_c_train)

In [ ]:
print('Accuracy :',accuracy_score(y_c_train, forest_c_best.predict(X_c_train)))
print('balanced_accuracy :',balanced_accuracy_score(y_c_train, forest_c_best.predict(X_c_train)))
print('precision :',precision_score(y_c_train, forest_c_best.predict(X_c_train), average='weighted'))
print('f1 :',f1_score(y_c_train, forest_c_best.predict(X_c_train), average='weighted'))
print('jaccard :',jaccard_score(y_c_train, forest_c_best.predict(X_c_train), average='weighted'))

Accuracy : 0.8192756062589615
balanced_accuracy : 0.4194440733082678
precision : 0.8514193027181808
f1 : 0.7573361718689015
jaccard : 0.672276095820634


Точностью (precision) называется доля правильных ответов модели в пределах класса — это доля объектов действительно принадлежащих данному классу относительно всех объектов которые система отнесла к этому классу.
Что нам и требуется.

SVM

In [27]:
svm_cc_param = {'kernel': ['linear', 'rbf'],
        'C': [5, 10]}
svm_cc_fit = SVC(random_state=21, probability=True)
svm_cc_grid = GridSearchCV(svm_cc_fit, svm_cc_param, scoring='precision', n_jobs=-1)
svm_cc_grid.fit(X_c_train, y_c_train)
svm_cc_grid.best_params_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  category=UserWarning,


{'C': 5, 'kernel': 'linear'}

In [32]:
svmc_best=SVC(random_state=21, kernel='linear', C=5)
svmc_best = svmc_best.fit(X_c_train, y_c_train)
print('precision :',precision_score(y_c_train, svmc_best.predict(X_c_train), average='weighted'))

precision : 0.7049690021588101


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


DecisionTree

In [33]:
dt_сc_param = {'max_depth': range(1, 50),
            'class_weight': ['balanced', None],
            'criterion': ['entropy', 'gini']}
dt_сc_fit = DecisionTreeClassifier(random_state=21)
dt_сc_grid = GridSearchCV(dt_сc_fit, dt_сc_param, scoring='precision', n_jobs=-1)
dt_сc_grid.fit(X_c_train, y_c_train)
dt_сc_grid.best_params_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  category=UserWarning,


{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 1}

In [34]:
dt_сc_best=DecisionTreeClassifier(random_state=21, class_weight='balanced', criterion='entropy', max_depth=1)
dt_сc_best =dt_сc_best.fit(X_c_train, y_c_train)
print('precision :',precision_score(y_c_train, dt_сc_best.predict(X_c_train), average='weighted'))

precision : 0.08270585931239312


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


LogisticRegression

In [35]:
lr_cc_params = {'fit_intercept': [True, False],
             'penalty': ['l1', 'l2', 'None'],
             'solver': ['liblinear', 'lbfgs']}
lr_cc_fit = LogisticRegression(random_state=21)     
lr_cc_grid = GridSearchCV(lr_cc_fit, lr_cc_params, scoring='precision', n_jobs=-1)     
lr_cc_grid.fit(X_c_train, y_c_train)
lr_cc_grid.best_params_  

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 449, in _check_solver
    % (solver, penalty)
ValueError: Solver

{'fit_intercept': True, 'penalty': 'l1', 'solver': 'liblinear'}

In [36]:
lr_cc_best=LogisticRegression(random_state=21, fit_intercept=True, penalty='l1', solver='liblinear')
lr_cc_best =lr_cc_best.fit(X_c_train, y_c_train)
print('precision :',precision_score(y_c_train, lr_cc_best.predict(X_c_train), average='weighted'))

precision : 0.708831687572659


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


RandomForest

In [37]:
rf_cc_params = {'n_estimators': [150, 200, 250],
             'max_depth': [10, 20, 30],
             'criterion': ['gini', 'entropy'],
             'class_weight': ['balanced', None]}
forest_cc_fit = RandomForestClassifier(random_state=21)
forest_cc_grid = GridSearchCV(forest_cc_fit, rf_cc_params, scoring='precision', n_jobs=-1)
forest_cc_grid.fit(X_c_train, y_c_train)
forest_cc_grid.best_params_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  category=UserWarning,


{'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 10,
 'n_estimators': 150}

In [38]:
rfc_best = RandomForestClassifier(random_state=21, class_weight='balanced', criterion='gini', max_depth=10, n_estimators=150)
rfc_best = rfc_best.fit(X_c_train, y_c_train)
print('precision :',precision_score(y_c_train, rfc_best.predict(X_c_train), average='weighted'))

precision : 0.7783002352893043


Лучшая модель RandomForestClassifier

In [ ]:
rfc_best = RandomForestClassifier(random_state=21, class_weight='balanced', criterion='gini', max_depth=10, n_estimators=150)
rfc_cc_best =rfc_best.fit(X_c_train, y_c_train)
print('precision :',precision_score(y_c_test, rfc_cc_best.predict(X_c_test), average='weighted'))

precision : 0.7244653230797761


Ансамбли

Voting

In [45]:
params = {'weights': [(1, 1, 1, 1), (2, 1, 1, 2), (1, 2, 1, 2), (1, 1, 2, 2), (2, 2, 1, 1), (1, 2, 2, 1), (2, 1, 2, 1), (3, 2, 1, 1), (1, 3, 2, 2), (2, 1, 3, 1), (3, 1, 2, 1)],
          'n_jobs': [-1]}

In [46]:
votreg_cc = VotingClassifier(estimators=[('LogisticRegression', lr_cc_best),('DecisionTreeClassifier', dt_сc_best),('RandomForestClassifier', rfc_best), ('SVC', svmc_best)])

In [47]:
Voti_cc=GridSearchCV(votreg_cc, params).fit(X_c_train, y_c_train) 

In [48]:
print('precision :',precision_score(y_c_train, Voti_cc.predict(X_c_train), average='weighted'))

precision : 0.7335898673857526


Bagging

In [51]:
params={'n_jobs':[8],
        'n_estimators': (10, 5)}

LogisticRegression

In [ ]:
bc_cc = BaggingClassifier(base_estimator = lr_cc_best)

In [ ]:
Baggi_cc = GridSearchCV(bc_cc, params).fit(X_c_train, y_c_train) 

In [ ]:
print('precision :',precision_score(y_c_train, Baggi_cc.predict(X_c_train), average='weighted'))

precision : 0.8143044896459479


RandomForest

In [ ]:
bc_rf_cc = BaggingClassifier(base_estimator = rfc_best)

In [ ]:
Baggi_rf_cc = GridSearchCV(bc_rf_cc, params).fit(X_c_train, y_c_train) 

In [ ]:
print('precision :',precision_score(y_c_train, Baggi_rf_cc.predict(X_c_train), average='weighted'))

precision : 0.7680701998281018


DecisionTree

In [ ]:
bc_dt_cc = BaggingClassifier(base_estimator = dt_сc_best)

In [ ]:
Baggi_dt_cc = GridSearchCV(bc_dt_cc, params).fit(X_c_train, y_c_train) 

In [ ]:
print('precision :',precision_score(y_c_train, Baggi_dt_cc.predict(X_c_train), average='weighted'))

precision : 0.08270585931239312


SVM

In [49]:
bc_svc_cc = BaggingClassifier(base_estimator = svmc_best)

In [53]:
Baggi_svc_cc = GridSearchCV(bc_svc_cc, params).fit(X_c_train, y_c_train) 

In [54]:
print('precision :',precision_score(y_c_train, Baggi_svc_cc.predict(X_c_train), average='weighted'))

precision : 0.8124430947610146


# **4. Принятие решения**

Наилучшие результаты у модели RandomForestClassifier

In [ ]:
rfc_cc_best=RandomForestClassifier(random_state=21,class_weight=None,criterion='gini',max_depth= 10,n_estimators=150)
rfc_cc_best=rfc_cc_best.fit(X_c_train, y_c_train)

In [ ]:
import joblib as joblib


In [ ]:
joblib.dump(rfc_cc_best, "/content/drive/MyDrive/School21/day15/Best_RandomForestClassifier.joblib")

['/content/drive/MyDrive/School21/day15/Best_RandomForestClassifier.joblib']

# Пищевая ценность

1. Соберите в датафрейм всю информацию о пищевой ценности продуктов из подготовленного вами и отфильтрованного набора данных (только столбцы с продуктами). Используйте для этого следующий API.
2. Конвертируйте все значения в % от суточной нормы потребления. Сохраняйте только те ингридиенты, которые находятся в этой и этой таблицах.
3. Сохраните измененный датафрейм в CSV-файл, который вы будете использовать в своей основной программе.

In [28]:
def search_for_nutritional_value(food_product):
    food_product =food_product
    url = f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key=24xx6p1JEOvUunxiWidkpN17IPgCPUFId73J5MkP&query={food_product}'
    requests.headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 YaBrowser/21.8.3.607 Yowser/2.5 Safari/537.36'}
    res = requests.get(url).json()
    Nutrients ={}
    try:
        x = res['foods'][0]['foodNutrients']
        for i in x:
            Nutrients[i['nutrientName']] = [i['value'], i['unitName']]
        return Nutrients
    except IndexError:
        Nutrients =0
        return Nutrients

In [29]:
def conversion_to_table(Nutrients,food_product):
    cc = pd.DataFrame(list(pp.items()),columns=['nutrition', 'value'])
    cc['measure'] = 0
    cc['food_product'] = food_product
    for i in range(len(cc)):
        x = cc.iloc[[i],[1]].value
        y = x[i]
        a = y[0]
        b = y[1]
        cc.iloc[[i],[1]] = a
        cc.iloc[[i],[2]] = b
        cc.iloc[[i],[3]] = food_product
    return cc

In [30]:
# новый список с названием столбцов
new_cols2 = [col.title() for col in df_narrowed.columns]
    
# меняем название
df_narrowed.columns = new_cols2
df_narrowed.columns

Index(['Title', 'Rating', 'Almond', 'Amaretto', 'Anchovy', 'Anise', 'Apple',
       'Apple Juice', 'Apricot', 'Artichoke',
       ...
       'Whiskey', 'White Wine', 'Whole Wheat', 'Wild Rice', 'Wine', 'Wok',
       'Yellow Squash', 'Yogurt', 'Yuca', 'Zucchini'],
      dtype='object', length=339)

In [31]:
df2 = pd.DataFrame(new_cols2)
df2

,0
0,Title
1,Rating
2,Almond
3,Amaretto
4,Anchovy
...,...
334,Wok
335,Yellow Squash
336,Yogurt
337,Yuca


In [32]:
df2.columns = df2.iloc[0]
df2 = df2[1:]

In [33]:
df2

,Title
1,Rating
2,Almond
3,Amaretto
4,Anchovy
5,Anise
...,...
334,Wok
335,Yellow Squash
336,Yogurt
337,Yuca


In [34]:
df2 = df2.drop(index=[
1
])

In [35]:
df2 = df2.reset_index(drop=
True
)

In [36]:
df2

,Title
0,Almond
1,Amaretto
2,Anchovy
3,Anise
4,Apple
...,...
332,Wok
333,Yellow Squash
334,Yogurt
335,Yuca


In [37]:
for i in range(len(df2)):
  x = df2['Title'].values[i]
  if i == 0:
    pp = search_for_nutritional_value(x)
    cc = conversion_to_table(pp, x)
    df_new1 = cc
  if i > 0:
    pp = search_for_nutritional_value(x)   
    if pp != 0:
      cc = conversion_to_table(pp, x)
      df_new1 = pd.concat([df_new1, cc])

In [38]:
df_new1

,nutrition,value,measure,food_product
0,Protein,21.0,G,Almond
1,Total lipid (fat),55.5,G,Almond
2,"Carbohydrate, by difference",18.8,G,Almond
3,Energy,614,KCAL,Almond
4,"Alcohol, ethyl",0.0,G,Almond
...,...,...,...,...
11,"Vitamin C, total ascorbic acid",12.6,MG,Zucchini
12,"Sugars, added",0.0,G,Zucchini
13,Cholesterol,0.0,MG,Zucchini
14,"Fatty acids, total trans",0.0,G,Zucchini


In [39]:
df_new1.to_csv('/content/drive/MyDrive/School21/day15/df_new.csv', index=False)

In [40]:
df_new1

,nutrition,value,measure,food_product
0,Protein,21.0,G,Almond
1,Total lipid (fat),55.5,G,Almond
2,"Carbohydrate, by difference",18.8,G,Almond
3,Energy,614,KCAL,Almond
4,"Alcohol, ethyl",0.0,G,Almond
...,...,...,...,...
11,"Vitamin C, total ascorbic acid",12.6,MG,Zucchini
12,"Sugars, added",0.0,G,Zucchini
13,Cholesterol,0.0,MG,Zucchini
14,"Fatty acids, total trans",0.0,G,Zucchini


In [41]:
!pip install tabula-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 6.2 MB/s 


In [42]:
import tabula

In [43]:
tabula.convert_into('/content/drive/MyDrive/School21/day15/Daily-Reference-Values-_DRVs_-under-the-New-NFL.pdf', output_format="csv", pages="all", output_path='/content/drive/MyDrive/School21/day15/Daily-Reference-Values-_DRVs_-under-the-New-NFL.csv')

Nov 10, 2022 2:48:48 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Nov 10, 2022 2:48:48 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>



In [44]:
df3 = pd.read_csv('/content/drive/MyDrive/School21/day15/Daily-Reference-Values-_DRVs_-under-the-New-NFL.csv')
df3

,DRVs - Food Components,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Food\rComponent,Unit of measure,Adults and Children ≥ 4\ryears,Infants through 12\rmonths,Children 1 through 3\ryears,Pregnant women and\rlactating women
1,Fat,Grams (g),178,30,239,178
2,Saturated fat,Grams (g),120,NaN,210,120
3,Cholesterol,Milligrams (mg),300,NaN,300,300
4,Total\rcarbohydrates,Grams (g),1275,95,2150,1275
5,Sodium,Milligrams (mg),"2,300",NaN,"1,500","2,300"
6,Dietary Fiber,Grams (g),128,NaN,214,128
7,Protein,Grams (g),150,NaN,213,NaN
8,Added sugars,Grams (g),150,NaN,225,150


In [45]:
df3.columns = df3.iloc[0]
df3 = df3[1:]
df3

,Food\rComponent,Unit of measure,Adults and Children ≥ 4\ryears,Infants through 12\rmonths,Children 1 through 3\ryears,Pregnant women and\rlactating women
1,Fat,Grams (g),178,30,239,178
2,Saturated fat,Grams (g),120,NaN,210,120
3,Cholesterol,Milligrams (mg),300,NaN,300,300
4,Total\rcarbohydrates,Grams (g),1275,95,2150,1275
5,Sodium,Milligrams (mg),"2,300",NaN,"1,500","2,300"
6,Dietary Fiber,Grams (g),128,NaN,214,128
7,Protein,Grams (g),150,NaN,213,NaN
8,Added sugars,Grams (g),150,NaN,225,150


In [46]:
tabula.convert_into('/content/drive/MyDrive/School21/day15/Reference-Daily-Intakes-_RDIs_-in-the-New-Nutrition-Facts-Label.pdf', output_format="csv", pages="all", output_path='/content/drive/MyDrive/School21/day15/Reference-Daily-Intakes-_RDIs_-in-the-New-Nutrition-Facts-Label.csv')

In [47]:
df3 = pd.read_csv('/content/drive/MyDrive/School21/day15/Daily-Reference-Values-_DRVs_-under-the-New-NFL.csv')
df3

,DRVs - Food Components,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Food\rComponent,Unit of measure,Adults and Children ≥ 4\ryears,Infants through 12\rmonths,Children 1 through 3\ryears,Pregnant women and\rlactating women
1,Fat,Grams (g),178,30,239,178
2,Saturated fat,Grams (g),120,NaN,210,120
3,Cholesterol,Milligrams (mg),300,NaN,300,300
4,Total\rcarbohydrates,Grams (g),1275,95,2150,1275
5,Sodium,Milligrams (mg),"2,300",NaN,"1,500","2,300"
6,Dietary Fiber,Grams (g),128,NaN,214,128
7,Protein,Grams (g),150,NaN,213,NaN
8,Added sugars,Grams (g),150,NaN,225,150


In [48]:
df3.columns = df3.iloc[0]
df3 = df3[1:]

In [49]:
df3

,Food\rComponent,Unit of measure,Adults and Children ≥ 4\ryears,Infants through 12\rmonths,Children 1 through 3\ryears,Pregnant women and\rlactating women
1,Fat,Grams (g),178,30,239,178
2,Saturated fat,Grams (g),120,NaN,210,120
3,Cholesterol,Milligrams (mg),300,NaN,300,300
4,Total\rcarbohydrates,Grams (g),1275,95,2150,1275
5,Sodium,Milligrams (mg),"2,300",NaN,"1,500","2,300"
6,Dietary Fiber,Grams (g),128,NaN,214,128
7,Protein,Grams (g),150,NaN,213,NaN
8,Added sugars,Grams (g),150,NaN,225,150


In [50]:
df3.columns = ['Food Component', 'Unit of measure', 'Adults and Children ≥ 4 years', 'Infants through 12 months', 'Children 1 through 3 years', 'Pregnant women and lactating women']
df3

,Food Component,Unit of measure,Adults and Children ≥ 4 years,Infants through 12 months,Children 1 through 3 years,Pregnant women and lactating women
1,Fat,Grams (g),178,30,239,178
2,Saturated fat,Grams (g),120,NaN,210,120
3,Cholesterol,Milligrams (mg),300,NaN,300,300
4,Total\rcarbohydrates,Grams (g),1275,95,2150,1275
5,Sodium,Milligrams (mg),"2,300",NaN,"1,500","2,300"
6,Dietary Fiber,Grams (g),128,NaN,214,128
7,Protein,Grams (g),150,NaN,213,NaN
8,Added sugars,Grams (g),150,NaN,225,150


In [51]:
tabula.convert_into('/content/drive/MyDrive/School21/day15/Reference-Daily-Intakes-_RDIs_-in-the-New-Nutrition-Facts-Label.pdf', output_format="csv", pages="all", output_path='/content/drive/MyDrive/School21/day15/Reference-Daily-Intakes-_RDIs_-in-the-New-Nutrition-Facts-Label.csv')

In [52]:
df4 = pd.read_csv('/content/drive/MyDrive/School21/day15/Reference-Daily-Intakes-_RDIs_-in-the-New-Nutrition-Facts-Label.csv')
df4

,RDIs - Nutrients,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Nutrient,Unit of measure,Adults and Children\r≥ 4 years,1Infants through 12\rmonths,Children 1 through 3\ryears,Pregnant women\rand lactating women
1,Vitamin A,Micrograms RAE2 (mcg),900,500,300,"1,300"
2,Vitamin C,Milligrams (mg),90,50,15,120
3,Calcium,Milligrams (mg),"1,300",260,700,"1,300"
4,Iron,Milligrams (mg),18,11,7,27
5,Vitamin D,Micrograms (mcg)3,20,10,15,15
6,Vitamin E,Milligrams (mg)4,15,5,6,19
7,Vitamin K,Micrograms (mcg),120,2.5,30,90
8,Thiamin,Milligrams (mg),1.2,0.3,0.5,1.4
9,Riboflavin,Milligrams (mg),1.3,0.4,0.5,1.6


In [53]:
df4.columns = df4.iloc[0]
df4 = df4[1:]
df4

,Nutrient,Unit of measure,Adults and Children\r≥ 4 years,1Infants through 12\rmonths,Children 1 through 3\ryears,Pregnant women\rand lactating women
1,Vitamin A,Micrograms RAE2 (mcg),900,500,300,"1,300"
2,Vitamin C,Milligrams (mg),90,50,15,120
3,Calcium,Milligrams (mg),"1,300",260,700,"1,300"
4,Iron,Milligrams (mg),18,11,7,27
5,Vitamin D,Micrograms (mcg)3,20,10,15,15
6,Vitamin E,Milligrams (mg)4,15,5,6,19
7,Vitamin K,Micrograms (mcg),120,2.5,30,90
8,Thiamin,Milligrams (mg),1.2,0.3,0.5,1.4
9,Riboflavin,Milligrams (mg),1.3,0.4,0.5,1.6
10,Niacin,Milligrams NE5 (mg),16,4,6,18


In [54]:
df5=df4.drop(labels = [28,29,30],axis = 0)

In [55]:
df5.columns = ['Food Component', 'Unit of measure', 'Adults and Children ≥ 4 years', 'Infants through 12 months', 'Children 1 through 3 years', 'Pregnant women and lactating women']
df5

,Food Component,Unit of measure,Adults and Children ≥ 4 years,Infants through 12 months,Children 1 through 3 years,Pregnant women and lactating women
1,Vitamin A,Micrograms RAE2 (mcg),900,500,300,"1,300"
2,Vitamin C,Milligrams (mg),90,50,15,120
3,Calcium,Milligrams (mg),"1,300",260,700,"1,300"
4,Iron,Milligrams (mg),18,11,7,27
5,Vitamin D,Micrograms (mcg)3,20,10,15,15
6,Vitamin E,Milligrams (mg)4,15,5,6,19
7,Vitamin K,Micrograms (mcg),120,2.5,30,90
8,Thiamin,Milligrams (mg),1.2,0.3,0.5,1.4
9,Riboflavin,Milligrams (mg),1.3,0.4,0.5,1.6
10,Niacin,Milligrams NE5 (mg),16,4,6,18


In [56]:
df6=pd.concat([df3, df5], ignore_index=True)
df6

,Food Component,Unit of measure,Adults and Children ≥ 4 years,Infants through 12 months,Children 1 through 3 years,Pregnant women and lactating women
0,Fat,Grams (g),178,30,239,178
1,Saturated fat,Grams (g),120,NaN,210,120
2,Cholesterol,Milligrams (mg),300,NaN,300,300
3,Total\rcarbohydrates,Grams (g),1275,95,2150,1275
4,Sodium,Milligrams (mg),"2,300",NaN,"1,500","2,300"
5,Dietary Fiber,Grams (g),128,NaN,214,128
6,Protein,Grams (g),150,NaN,213,NaN
7,Added sugars,Grams (g),150,NaN,225,150
8,Vitamin A,Micrograms RAE2 (mcg),900,500,300,"1,300"
9,Vitamin C,Milligrams (mg),90,50,15,120


In [57]:
df6.to_excel('/content/drive/MyDrive/School21/day15/vitamin.xlsx')

In [58]:
vit = pd.DataFrame(pd.read_excel('/content/drive/MyDrive/School21/day15/food_nut.xlsx'))
vit.to_csv('/content/drive/MyDrive/School21/day15/food_nut.csv', index = False)
vit

,ndex,Nutrient,Unit,Norm,food_nut
0,0,Fat,Grams (g),78.0,Total lipid (fat)
1,1,Saturated fat,Grams (g),20.0,"Fatty acids, total saturated"
2,2,Cholesterol,Milligrams (mg),300.0,Cholesterol
3,3,Total carbohydrates,Grams (g),275.0,"Carbohydrate, by difference"
4,4,Sodium,Milligrams (mg),2300.0,"Sodium, Na"
5,5,Dietary Fiber,Grams (g),28.0,"Fiber, total dietary"
6,6,Protein,Grams (g),50.0,Protein
7,7,Added sugars,Grams (g),50.0,"Sugars, added"
8,8,Vitamin A,Micrograms RAE2 (mcg),900.0,"Vitamin A, RAE"
9,9,Vitamin C,Milligrams (mg),90.0,"Vitamin C, total ascorbic acid"


In [59]:
df_new1

,nutrition,value,measure,food_product
0,Protein,21.0,G,Almond
1,Total lipid (fat),55.5,G,Almond
2,"Carbohydrate, by difference",18.8,G,Almond
3,Energy,614,KCAL,Almond
4,"Alcohol, ethyl",0.0,G,Almond
...,...,...,...,...
11,"Vitamin C, total ascorbic acid",12.6,MG,Zucchini
12,"Sugars, added",0.0,G,Zucchini
13,Cholesterol,0.0,MG,Zucchini
14,"Fatty acids, total trans",0.0,G,Zucchini


In [60]:
df_new3=df_new1.merge(vit, left_on='nutrition', right_on='food_nut')
df_new3

,nutrition,value,measure,food_product,ndex,Nutrient,Unit,Norm,food_nut
0,Protein,21.0,G,Almond,6,Protein,Grams (g),50.0,Protein
1,Protein,0.0,G,Amaretto,6,Protein,Grams (g),50.0,Protein
2,Protein,28.9,G,Anchovy,6,Protein,Grams (g),50.0,Protein
3,Protein,17.6,G,Anise,6,Protein,Grams (g),50.0,Protein
4,Protein,0.0,G,Apple,6,Protein,Grams (g),50.0,Protein
...,...,...,...,...,...,...,...,...,...
5563,"Manganese, Mn",0.137,MG,Wasabi,25,Magnesium,Milligrams (mg),420.0,"Manganese, Mn"
5564,"Manganese, Mn",0.137,MG,Wasabi,29,Manganese,Milligrams (mg),2.3,"Manganese, Mn"
5565,"Manganese, Mn",0.0,MG,Wok,25,Magnesium,Milligrams (mg),420.0,"Manganese, Mn"
5566,"Manganese, Mn",0.0,MG,Wok,29,Manganese,Milligrams (mg),2.3,"Manganese, Mn"


In [61]:
df_new3['value']=df_new3['value'].astype(int)

In [62]:
df_new3['value2']=df_new3['value'].values/df_new3['Norm'].values# находим % от суточной нормы потребления
df_new3

,nutrition,value,measure,food_product,ndex,Nutrient,Unit,Norm,food_nut,value2
0,Protein,21,G,Almond,6,Protein,Grams (g),50.0,Protein,0.42
1,Protein,0,G,Amaretto,6,Protein,Grams (g),50.0,Protein,0.00
2,Protein,28,G,Anchovy,6,Protein,Grams (g),50.0,Protein,0.56
3,Protein,17,G,Anise,6,Protein,Grams (g),50.0,Protein,0.34
4,Protein,0,G,Apple,6,Protein,Grams (g),50.0,Protein,0.00
...,...,...,...,...,...,...,...,...,...,...
5563,"Manganese, Mn",0,MG,Wasabi,25,Magnesium,Milligrams (mg),420.0,"Manganese, Mn",0.00
5564,"Manganese, Mn",0,MG,Wasabi,29,Manganese,Milligrams (mg),2.3,"Manganese, Mn",0.00
5565,"Manganese, Mn",0,MG,Wok,25,Magnesium,Milligrams (mg),420.0,"Manganese, Mn",0.00
5566,"Manganese, Mn",0,MG,Wok,29,Manganese,Milligrams (mg),2.3,"Manganese, Mn",0.00


In [63]:
df_new3.drop(columns={"Nutrient","Unit","ndex","Norm","food_nut",'value'} , axis=1, inplace=True)

In [64]:
df_new3.rename(columns={'value2': 'value'}, inplace=True)
df_new3


,nutrition,measure,food_product,value
0,Protein,G,Almond,0.42
1,Protein,G,Amaretto,0.00
2,Protein,G,Anchovy,0.56
3,Protein,G,Anise,0.34
4,Protein,G,Apple,0.00
...,...,...,...,...
5563,"Manganese, Mn",MG,Wasabi,0.00
5564,"Manganese, Mn",MG,Wasabi,0.00
5565,"Manganese, Mn",MG,Wok,0.00
5566,"Manganese, Mn",MG,Wok,0.00


In [ ]:
df_new3.to_csv("/content/drive/MyDrive/School21/day15/nutrition.csv",index=False)

In [65]:
df_new5 = pd.read_csv('../content/drive/MyDrive/School21/day15/nutrition.csv')
df_new5

,nutrition,measure,food_product,value
0,Protein,G,Almond,0.42
1,Protein,G,Amaretto,0.00
2,Protein,G,Anchovy,0.56
3,Protein,G,Anise,0.34
4,Protein,G,Apple,0.00
...,...,...,...,...
5563,"Manganese, Mn",MG,Wasabi,0.00
5564,"Manganese, Mn",MG,Wasabi,0.00
5565,"Manganese, Mn",MG,Wok,0.00
5566,"Manganese, Mn",MG,Wok,0.00


# **Похожие рецепты**

1. Для каждого рецепта из набора данных найдите ссылку на сайте epicurious.com и подробную информацию о нем (название рецепта, рейтинг на платформе и URl). Если сделать это не удалось, найдите для данного рецепта похожую ссылку в Интернете.
2. Сохраните новый датафрейм в CSV-файл, который вы будете использовать в своей основной программе.

In [110]:
pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [111]:
import requests

In [112]:
import unidecode
from bs4 import BeautifulSoup
import re

In [113]:

# Получим URL из названия рецепта
def get_recipe_url_epic(recipe):
    '''
    Функция создаёт URL рецепта с https://www.epicurious.com/ на основании названия рецепта в датафрейме 
    '''
    list_of_ingredients = re.findall(r'\b\w+\b', recipe)
    url0 = 'https://www.epicurious.com/search/'
    url = url0 + "%2C%20".join(list_of_ingredients)
    return url

recipe_name_list = df_narrowed['Title'].tolist()
recipe_list = []
for recipe in recipe_name_list:
    recipe_list.append(get_recipe_url_epic(recipe))
df_narrowed["URL"] = recipe_list
df_narrowed

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,Title,Rating,Almond,Amaretto,Anchovy,Anise,Apple,Apple Juice,Apricot,Artichoke,...,White Wine,Whole Wheat,Wild Rice,Wine,Wok,Yellow Squash,Yogurt,Yuca,Zucchini,URL
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Lentil%2C%20...
1,Boudin Blanc Terrine with Red Onion Confit,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Boudin%2C%20...
2,Potato and Fennel Soup Hodge,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Potato%2C%20...
3,Mahi-Mahi in Tomato Olive Sauce,5.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Mahi%2C%20Ma...
4,Spinach Noodle Casserole,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Spinach%2C%2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Parmesan%2C%...
20048,Artichoke and Parmesan Risotto,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Artichoke%2C...
20049,Turkey Cream Puff Pie,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Turkey%2C%20...
20050,Snapper on Angel Hair with Citrus Cream,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Snapper%2C%2...


In [116]:
df_new['val'] = df_new.sum (axis=1) 

In [115]:
df_new

,Almond,Amaretto,Anchovy,Anise,Apple,Apple Juice,Apricot,Artichoke,Arugula,Asian Pear,...,White Wine,Whole Wheat,Wild Rice,Wine,Wok,Yellow Squash,Yogurt,Yuca,Zucchini,val
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
20048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
20049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
20050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0


In [117]:
val = df_new[['val']]
val

,val
0,12.0
1,10.0
2,6.0
3,6.0
4,6.0
...,...
20047,6.0
20048,4.0
20049,6.0
20050,12.0


In [118]:
df7 = pd.concat([df_narrowed, val.reindex(df_narrowed.index)], axis=1)
df7

,Title,Rating,Almond,Amaretto,Anchovy,Anise,Apple,Apple Juice,Apricot,Artichoke,...,Whole Wheat,Wild Rice,Wine,Wok,Yellow Squash,Yogurt,Yuca,Zucchini,URL,val
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Lentil%2C%20...,12.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Boudin%2C%20...,10.0
2,Potato and Fennel Soup Hodge,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Potato%2C%20...,6.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Mahi%2C%20Ma...,6.0
4,Spinach Noodle Casserole,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Spinach%2C%2...,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Parmesan%2C%...,6.0
20048,Artichoke and Parmesan Risotto,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Artichoke%2C...,4.0
20049,Turkey Cream Puff Pie,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Turkey%2C%20...,6.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Snapper%2C%2...,12.0


In [119]:
df = pd.read_csv('/content/drive/MyDrive/School21/day15/epi_r.csv')

In [120]:
df10 = df[[
'dinner', 'breakfast', 'lunch'
, 
'title'
]]
df10

,dinner,breakfast,lunch,title
0,0.0,0.0,0.0,"Lentil, Apple, and Turkey Wrap"
1,0.0,0.0,0.0,Boudin Blanc Terrine with Red Onion Confit
2,0.0,0.0,0.0,Potato and Fennel Soup Hodge
3,1.0,0.0,0.0,Mahi-Mahi in Tomato Olive Sauce
4,0.0,0.0,0.0,Spinach Noodle Casserole
...,...,...,...,...
20047,0.0,0.0,0.0,Parmesan Puffs
20048,1.0,0.0,0.0,Artichoke and Parmesan Risotto
20049,1.0,0.0,0.0,Turkey Cream Puff Pie
20050,0.0,0.0,0.0,Snapper on Angel Hair with Citrus Cream


In [121]:
df10 = df10.rename(columns={
'title'
: 
'Title'})
df10

,dinner,breakfast,lunch,Title
0,0.0,0.0,0.0,"Lentil, Apple, and Turkey Wrap"
1,0.0,0.0,0.0,Boudin Blanc Terrine with Red Onion Confit
2,0.0,0.0,0.0,Potato and Fennel Soup Hodge
3,1.0,0.0,0.0,Mahi-Mahi in Tomato Olive Sauce
4,0.0,0.0,0.0,Spinach Noodle Casserole
...,...,...,...,...
20047,0.0,0.0,0.0,Parmesan Puffs
20048,1.0,0.0,0.0,Artichoke and Parmesan Risotto
20049,1.0,0.0,0.0,Turkey Cream Puff Pie
20050,0.0,0.0,0.0,Snapper on Angel Hair with Citrus Cream


In [122]:
df18 = pd.merge(df10, df7)
df18

,dinner,breakfast,lunch,Title,Rating,Almond,Amaretto,Anchovy,Anise,Apple,...,Whole Wheat,Wild Rice,Wine,Wok,Yellow Squash,Yogurt,Yuca,Zucchini,URL,val
0,0.0,0.0,0.0,"Lentil, Apple, and Turkey Wrap",2.500,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Lentil%2C%20...,12.0
1,0.0,0.0,0.0,Boudin Blanc Terrine with Red Onion Confit,4.375,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Boudin%2C%20...,10.0
2,0.0,0.0,0.0,Potato and Fennel Soup Hodge,3.750,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Potato%2C%20...,6.0
3,1.0,0.0,0.0,Mahi-Mahi in Tomato Olive Sauce,5.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Mahi%2C%20Ma...,6.0
4,0.0,0.0,0.0,Spinach Noodle Casserole,3.125,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Spinach%2C%2...,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27899,0.0,0.0,0.0,Chinese Barbecued Spareribs,3.750,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Chinese%2C%2...,8.0
27900,1.0,0.0,0.0,Artichoke and Parmesan Risotto,4.375,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Artichoke%2C...,4.0
27901,1.0,0.0,0.0,Turkey Cream Puff Pie,4.375,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Turkey%2C%20...,6.0
27902,0.0,0.0,0.0,Snapper on Angel Hair with Citrus Cream,4.375,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Snapper%2C%2...,12.0


In [123]:
df18['meal'] =df18.apply(lambda x: '%s%s%s' % (x['breakfast'],x['lunch'],x['dinner']),axis=1)

In [124]:

# дополнительно распределяем рецепты, которые едят одновременно на разные случаи
df18['meal']=df18['meal'].replace('0.00.01.0','dinner')
df18['meal']=df18['meal'].replace('0.00.01.0','dinner')
df18['meal']=df18['meal'].replace('1.00.00.0','breakfast')
df18['meal']=df18['meal'].replace('1.00.01.0','breakfast')
df18['meal']=df18['meal'].replace('1.01.01.0','breakfast')
df18['meal']=df18['meal'].replace('1.01.00.0','breakfast')
df18['meal']=df18['meal'].replace('0.01.00.0','lunch')
df18['meal']=df18['meal'].replace('0.01.01.0','lunch')
df18['meal']=df18['meal'].replace('0.00.00.0','other')
df18.drop(columns={"breakfast","lunch","dinner"} , axis=1, inplace=True)

In [125]:
df18['class']=round(df18['Rating'],0).astype(int)
df18['class']=df18['class'].replace(0,'bad')
df18['class']=df18['class'].replace(1,'bad')
df18['class']=df18['class'].replace(2,'so-so')
df18['class']=df18['class'].replace(3,'so-so')
df18['class']=df18['class'].replace(4,'great')
df18['class']=df18['class'].replace(5,'great')

In [126]:
df18


,Title,Rating,Almond,Amaretto,Anchovy,Anise,Apple,Apple Juice,Apricot,Artichoke,...,Wine,Wok,Yellow Squash,Yogurt,Yuca,Zucchini,URL,val,meal,class
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Lentil%2C%20...,12.0,other,so-so
1,Boudin Blanc Terrine with Red Onion Confit,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Boudin%2C%20...,10.0,other,great
2,Potato and Fennel Soup Hodge,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Potato%2C%20...,6.0,other,great
3,Mahi-Mahi in Tomato Olive Sauce,5.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Mahi%2C%20Ma...,6.0,dinner,great
4,Spinach Noodle Casserole,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Spinach%2C%2...,6.0,other,so-so
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27899,Chinese Barbecued Spareribs,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Chinese%2C%2...,8.0,other,great
27900,Artichoke and Parmesan Risotto,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Artichoke%2C...,4.0,dinner,great
27901,Turkey Cream Puff Pie,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Turkey%2C%20...,6.0,dinner,great
27902,Snapper on Angel Hair with Citrus Cream,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Snapper%2C%2...,12.0,other,great


In [127]:
df18.to_csv('/content/drive/MyDrive/School21/day15/recipes.csv',index = False)

In [128]:
df_result = pd.read_csv('/content/drive/MyDrive/School21/day15/recipes.csv')
df_result

,Title,Rating,Almond,Amaretto,Anchovy,Anise,Apple,Apple Juice,Apricot,Artichoke,...,Wine,Wok,Yellow Squash,Yogurt,Yuca,Zucchini,URL,val,meal,class
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Lentil%2C%20...,12.0,other,so-so
1,Boudin Blanc Terrine with Red Onion Confit,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Boudin%2C%20...,10.0,other,great
2,Potato and Fennel Soup Hodge,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Potato%2C%20...,6.0,other,great
3,Mahi-Mahi in Tomato Olive Sauce,5.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Mahi%2C%20Ma...,6.0,dinner,great
4,Spinach Noodle Casserole,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Spinach%2C%2...,6.0,other,so-so
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27899,Chinese Barbecued Spareribs,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Chinese%2C%2...,8.0,other,great
27900,Artichoke and Parmesan Risotto,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Artichoke%2C...,4.0,dinner,great
27901,Turkey Cream Puff Pie,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Turkey%2C%20...,6.0,dinner,great
27902,Snapper on Angel Hair with Citrus Cream,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/search/Snapper%2C%2...,12.0,other,great
